In [2]:
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

import warnings
warnings.filterwarnings('ignore')

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.externals import joblib
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from my_transformer import change_to_datetime, divide_datetime, add_dayofweek, feature_selection, one_hot_encoding, standard_scaler, concat, drop_feature

In [4]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR

In [5]:
knn = KNeighborsRegressor(n_neighbors=5)
linear = LinearRegression()
sgd = SGDRegressor(max_iter=None, eta0=0.01, penalty='l2', random_state=30)
ridge = Ridge(alpha=1, random_state=30)
lasso = Lasso(alpha=1, random_state=30)
elastic = ElasticNet(alpha=1, l1_ratio=0.5, random_state=30)
dt = DecisionTreeRegressor(max_depth=None, random_state=30)
rf = RandomForestRegressor(n_estimators=100, max_depth=None, random_state=30, n_jobs=-1)
gb = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=30)
svm = SVR(C=1, kernel='rbf', gamma='auto_deprecated')

In [6]:
from sklearn.metrics import make_scorer

def rmsle(predicted_values, actual_values):
    # 넘파이로 배열 형태로 바꿔준다.
    predicted_values = np.array(predicted_values)
    actual_values = np.array(actual_values)
    
    # 예측값과 실제 값에 1을 더하고 로그를 씌워준다.
    log_predict = np.log(predicted_values + 1)
    log_actual = np.log(actual_values + 1)
    
    # 위에서 계산한 예측값에서 실제값을 빼주고 제곱을 해준다.
    difference = log_predict - log_actual
    # difference = (log_predict - log_actual) ** 2
    difference = np.square(difference)
    
    # 평균을 낸다.
    mean_difference = difference.mean()
    
    # 다시 루트를 씌운다.
    score = np.sqrt(mean_difference)
    
    return score

rmsle_scorer = make_scorer(rmsle)

In [7]:
from sklearn.metrics import make_scorer

def neg_rmsle(predicted_values, actual_values):
    # 넘파이로 배열 형태로 바꿔준다.
    predicted_values = np.array(predicted_values)
    actual_values = np.array(actual_values)
    
    # 예측값과 실제 값에 1을 더하고 로그를 씌워준다.
    log_predict = np.log(predicted_values + 1)
    log_actual = np.log(actual_values + 1)
    
    # 위에서 계산한 예측값에서 실제값을 빼주고 제곱을 해준다.
    difference = log_predict - log_actual
    # difference = (log_predict - log_actual) ** 2
    difference = np.square(difference)
    
    # 평균을 낸다.
    mean_difference = difference.mean()
    
    # 다시 루트를 씌운다.
    score = np.sqrt(mean_difference) * (-1)
    
    return score

neg_rmsle_scorer = make_scorer(neg_rmsle)

In [8]:
preparation = joblib.load("preparation.pkl")
pipeline_cat = joblib.load("pipeline_cat.pkl")
pipeline_num = joblib.load("pipeline_num.pkl")

In [9]:
train = pd.read_csv('train.csv')
# train = pd.read_csv("data/train.csv", parse_dates=["datetime"]) 와 같이 불러오면 data type을 변경할 필요 없음
test = pd.read_csv('test.csv')

In [10]:
data = train.drop(['casual', 'registered', 'count'], axis=1)
y = train['count']

In [11]:
data = preparation.transform(data)
data.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,month,day,hour,dayofweek
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,2011,1,1,0,5
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,2011,1,1,1,5
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,2011,1,1,2,5
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,2011,1,1,3,5
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,2011,1,1,4,5


In [12]:
feature_cat = ['holiday', 'workingday', 'weather', 'year', 'month', 'hour', 'dayofweek']
feature_num = ['temp', 'humidity', 'windspeed']

In [13]:
pipeline_cat = make_pipeline(
    feature_selection(feature_cat),
    one_hot_encoding('weather', 'weather'),
    one_hot_encoding('hour', 'hour'),
)
joblib.dump(pipeline_cat, "pipeline_cat.pkl")

['pipeline_cat.pkl']

In [14]:
X_cat = pipeline_cat.transform(data)
X_num = pipeline_num.transform(data)
X = concat(X_cat, X_num)

In [15]:
remove = ['dayofweek', 'weather_4', 'month']
drop = drop_feature(remove)
X = drop.transform(X)

In [16]:
param_grid = [
    {'max_features': [24, 28, 32], 'n_estimators': [900]},
  ]

rf= RandomForestRegressor(random_state=30)
search = GridSearchCV(rf, param_grid, cv=5, scoring=neg_rmsle_scorer, return_train_score=True, n_jobs=-1)
search.fit(X, y)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=30, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid=[{'max_features': [24, 28, 32], 'n_estimators': [900]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=make_scorer(neg_rmsle), verbose=0)

In [17]:
search.best_params_

{'max_features': 32, 'n_estimators': 900}

In [18]:
result = search.cv_results_
for mean_score, params in zip(result["mean_test_score"], result["params"]):
    print('{} : {:.4f}' .format(params, abs(mean_score)))

{'max_features': 24, 'n_estimators': 900} : 0.6811
{'max_features': 28, 'n_estimators': 900} : 0.6681
{'max_features': 32, 'n_estimators': 900} : 0.6591


In [19]:
feature_importances = search.best_estimator_.feature_importances_
sorted(zip(feature_importances, X.columns), reverse=True)

[(0.16929696750029924, 'temp'),
 (0.13440214550994975, 'humidity'),
 (0.09872800766018516, 'workingday'),
 (0.09346078788703452, 'hour_17'),
 (0.08008808604067845, 'year'),
 (0.07795301755067918, 'hour_18'),
 (0.06302950131083353, 'hour_8'),
 (0.031235863202507496, 'hour_19'),
 (0.025293423588320207, 'windspeed'),
 (0.018764596758478788, 'hour_7'),
 (0.016346231313272994, 'hour_1'),
 (0.01593597682764304, 'hour_2'),
 (0.015303604027429767, 'hour_16'),
 (0.014819018309183815, 'hour_3'),
 (0.014726518024712427, 'hour_0'),
 (0.014447714408060182, 'hour_4'),
 (0.013077974577166047, 'hour_5'),
 (0.012063121368711113, 'hour_20'),
 (0.0115609216214557, 'hour_9'),
 (0.010870389785739645, 'hour_23'),
 (0.010721478765615307, 'weather_3'),
 (0.007855598145151365, 'hour_6'),
 (0.006997398031246716, 'hour_22'),
 (0.006318288709226374, 'hour_21'),
 (0.005520378051561744, 'hour_10'),
 (0.005349493893835901, 'hour_12'),
 (0.004582644632968138, 'hour_11'),
 (0.004581714742804624, 'hour_13'),
 (0.003723